In this project we will be working with Movie Lens dataset and will try to build a recomendation system. 

# Importing all essential libraries

In [1]:
import os
import sys


os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

#import required libraries
import csv
import matplotlib.pyplot as plt
import gmplot
import pandas as pd
import numpy as np
import pyspark
import seaborn as sns
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import col, lit, when
# Import libraries and other functions
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum, col, desc

# ML lib
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.sql.functions import explode

from IPython.display import display
from IPython.display import Image


# Creating spark session

In [2]:
spark=SparkSession.builder.appName("Recomendation").getOrCreate()


# Importing csv files into Jupyter notebook from HDFS --START

In [3]:
#importing rating dataset
df_ratings= spark.read.csv('hdfs:///user/imat5322_376434/ratings.csv',header=True,inferSchema=True)

In [4]:
#importing movie dataset

df_movies=spark.read.csv('hdfs:///user/imat5322_376434/movies.csv',header=True,inferSchema=True)

In [5]:
#importing links dataset

df_links=spark.read.csv('hdfs:///user/imat5322_376434/links.csv',header=True,inferSchema=True)

In [6]:
#importing movie metadata dataset - additional metadata file

df_movies_metadata=spark.read.csv('hdfs:///user/imat5322_376434/movies_metadata.csv',header=True,inferSchema=True)

In [7]:
#importing language code dataset - additional metadata file

df_language_codes=spark.read.csv('hdfs:///user/imat5322_376434/language_codes.csv',header=True,inferSchema=True)

# Import Pixiedust for visualization

In [8]:
from pixiedust.display import *

Pixiedust database opened successfully


# Initial data exploration

Movie dataset

In [9]:
display(df_links.describe())

In [10]:
#df_movies.show()

In [11]:
#df_movies.count()

In [12]:

#initialize Counter to 0
i = 0 

#Total records in the df_movies dataframe

total = df_movies.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(df_movies.columns)))
print("+++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++")

#while Loop used here to display the count of  each column's missing values 

while i < len(df_movies.columns):                             
#Loop which checks every column
    print(str(i+1) + "." + str(df_movies[i]))
    print("  Missing Values = ") 
   
    missingvalue1 = df_movies.select([count(when(df_movies[i].isNull(),True))]).show() 
    i = i+1 
    #Counter is incremented by 1 

Total Records = 58098
Total columns = 3
+++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++
1.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

2.Column<b'title'>
  Missing Values = 
+----------------------------------------------+
|count(CASE WHEN (title IS NULL) THEN true END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+

3.Column<b'genres'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (genres IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+



Rating dataset

In [13]:
#df_ratings.show()

In [14]:
df_movies_metadata.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [15]:
#display(df_language_codes.describe())

In [16]:
#df_ratings.count()

In [17]:

#initialize Counter to 0
i = 0 

#Total records in the df_ratings dataframe

total = df_ratings.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(df_ratings.columns)))
print("+++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++")

# while Loop used here to display the count of  
# each column's missing values 

while i < len(df_ratings.columns):                             
#Loop which checks every column
    print(str(i+1) + "." + str(df_ratings[i]))
    print("  Missing Values = ") 
   
    missingvalue1 = df_ratings \
    .select([count(when(df_ratings[i].isNull(),True))]).show() 
    i = i+1 
    #Counter is incremented by 1

Total Records = 9139592
Total columns = 4
+++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++
1.Column<b'userId'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (userId IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

2.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

3.Column<b'rating'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (rating IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+---------------------------------------------

Movies metadata dataset

In [18]:
#df_movies_metadata.show()

In [19]:
#df_movies_metadata.count()

In [20]:

#initialize Counter to 0
i = 0 

#Total records in the df_ratings dataframe

total = df_language_codes.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(df_language_codes.columns)))
print("+++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++")

# while Loop used here to display the count of  
# each column's missing values 

while i < len(df_language_codes.columns):                             
#Loop which checks every column
    print(str(i+1) + "." + str(df_ratings[i]))
    print("  Missing Values = ") 
   
    missingvalue1 = df_language_codes \
    .select([count(when(df_language_codes[i].isNull(),True))]).show() 
    i = i+1 
    #Counter is incremented by 1

Total Records = 184
Total columns = 2
+++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++
1.Column<b'userId'>
  Missing Values = 
+---------------------------------------------+
|count(CASE WHEN (code IS NULL) THEN true END)|
+---------------------------------------------+
|                                            0|
+---------------------------------------------+

2.Column<b'movieId'>
  Missing Values = 
+-------------------------------------------------+
|count(CASE WHEN (language IS NULL) THEN true END)|
+-------------------------------------------------+
|                                                0|
+-------------------------------------------------+



Currently we are not working with df_language_code dataframe. We will use it at appropriate place. This dataset is small and accurate.

# Data cleaning and pre-prossesing

Cleaning and pre-processing of df_ratings dataframe

In [21]:
df_ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [22]:
#converting timestamp to actual date

df_ratings = (df_ratings.select("userId","movieId","rating","timestamp") \
              .withColumn("Date_of_rating", from_unixtime("timestamp")))
df_ratings.show()

+------+-------+------+----------+-------------------+
|userId|movieId|rating| timestamp|     Date_of_rating|
+------+-------+------+----------+-------------------+
|     1|    307|   3.5|1256677221|2009-10-27 21:00:21|
|     1|    481|   3.5|1256677456|2009-10-27 21:04:16|
|     1|   1091|   1.5|1256677471|2009-10-27 21:04:31|
|     1|   1257|   4.5|1256677460|2009-10-27 21:04:20|
|     1|   1449|   4.5|1256677264|2009-10-27 21:01:04|
|     1|   1590|   2.5|1256677236|2009-10-27 21:00:36|
|     1|   1591|   1.5|1256677475|2009-10-27 21:04:35|
|     1|   2134|   4.5|1256677464|2009-10-27 21:04:24|
|     1|   2478|   4.0|1256677239|2009-10-27 21:00:39|
|     1|   2840|   3.0|1256677500|2009-10-27 21:05:00|
|     1|   2986|   2.5|1256677496|2009-10-27 21:04:56|
|     1|   3020|   4.0|1256677260|2009-10-27 21:01:00|
|     1|   3424|   4.5|1256677444|2009-10-27 21:04:04|
|     1|   3698|   3.5|1256677243|2009-10-27 21:00:43|
|     1|   3826|   2.0|1256677210|2009-10-27 21:00:10|
|     1|  

In [23]:
#Extracting the year,month, date, hour data from the Date_of_rating

df_ratings_final = df_ratings \
.withColumn("year",expr("substring(Date_of_rating, 1, 4)")) \
.withColumn("month",expr("substring(Date_of_rating, 6, 2)")) \
.withColumn("date",expr("substring(Date_of_rating, 9, 2)")) \
.withColumn("hour",expr("substring(Date_of_rating, 12, 2)"))



In [24]:
df_ratings_final.show()

+------+-------+------+----------+-------------------+----+-----+----+----+
|userId|movieId|rating| timestamp|     Date_of_rating|year|month|date|hour|
+------+-------+------+----------+-------------------+----+-----+----+----+
|     1|    307|   3.5|1256677221|2009-10-27 21:00:21|2009|   10|  27|  21|
|     1|    481|   3.5|1256677456|2009-10-27 21:04:16|2009|   10|  27|  21|
|     1|   1091|   1.5|1256677471|2009-10-27 21:04:31|2009|   10|  27|  21|
|     1|   1257|   4.5|1256677460|2009-10-27 21:04:20|2009|   10|  27|  21|
|     1|   1449|   4.5|1256677264|2009-10-27 21:01:04|2009|   10|  27|  21|
|     1|   1590|   2.5|1256677236|2009-10-27 21:00:36|2009|   10|  27|  21|
|     1|   1591|   1.5|1256677475|2009-10-27 21:04:35|2009|   10|  27|  21|
|     1|   2134|   4.5|1256677464|2009-10-27 21:04:24|2009|   10|  27|  21|
|     1|   2478|   4.0|1256677239|2009-10-27 21:00:39|2009|   10|  27|  21|
|     1|   2840|   3.0|1256677500|2009-10-27 21:05:00|2009|   10|  27|  21|
|     1|   2

In [25]:
#Converting the date to day of the week

df_ratings_final=df_ratings_final.withColumn("week_day_abb", date_format(col("Date_of_rating"), "E"))


In [26]:
#dropping timestamp and Date_of_rating because they are not required anymore

df_ratings_final=df_ratings_final.drop("timestamp").drop("Date_of_rating")
df_ratings_final.show()

+------+-------+------+----+-----+----+----+------------+
|userId|movieId|rating|year|month|date|hour|week_day_abb|
+------+-------+------+----+-----+----+----+------------+
|     1|    307|   3.5|2009|   10|  27|  21|         Tue|
|     1|    481|   3.5|2009|   10|  27|  21|         Tue|
|     1|   1091|   1.5|2009|   10|  27|  21|         Tue|
|     1|   1257|   4.5|2009|   10|  27|  21|         Tue|
|     1|   1449|   4.5|2009|   10|  27|  21|         Tue|
|     1|   1590|   2.5|2009|   10|  27|  21|         Tue|
|     1|   1591|   1.5|2009|   10|  27|  21|         Tue|
|     1|   2134|   4.5|2009|   10|  27|  21|         Tue|
|     1|   2478|   4.0|2009|   10|  27|  21|         Tue|
|     1|   2840|   3.0|2009|   10|  27|  21|         Tue|
|     1|   2986|   2.5|2009|   10|  27|  21|         Tue|
|     1|   3020|   4.0|2009|   10|  27|  21|         Tue|
|     1|   3424|   4.5|2009|   10|  27|  21|         Tue|
|     1|   3698|   3.5|2009|   10|  27|  21|         Tue|
|     1|   382

In [27]:
df_ratings_final = df_ratings_final. \
withColumn("month", regexp_replace("month" , "01" ,"Jan")). \
withColumn("month", regexp_replace("month" , "02" ,"Feb")). \
withColumn("month", regexp_replace("month" , "03" ,"Mar")). \
withColumn("month", regexp_replace("month" , "04" ,"Apr")). \
withColumn("month", regexp_replace("month" , "05" ,"May")). \
withColumn("month", regexp_replace("month" , "06" ,"Jun")). \
withColumn("month", regexp_replace("month" , "07" ,"Jul")). \
withColumn("month", regexp_replace("month" , "08" ,"Aug")). \
withColumn("month", regexp_replace("month" , "09" ,"Sep")). \
withColumn("month", regexp_replace("month" , "10" ,"Oct")). \
withColumn("month", regexp_replace("month" , "11" ,"Nov")). \
withColumn("month", regexp_replace("month" , "12" ,"Dec"))



In [28]:
df_ratings_final.show()

+------+-------+------+----+-----+----+----+------------+
|userId|movieId|rating|year|month|date|hour|week_day_abb|
+------+-------+------+----+-----+----+----+------------+
|     1|    307|   3.5|2009|  Oct|  27|  21|         Tue|
|     1|    481|   3.5|2009|  Oct|  27|  21|         Tue|
|     1|   1091|   1.5|2009|  Oct|  27|  21|         Tue|
|     1|   1257|   4.5|2009|  Oct|  27|  21|         Tue|
|     1|   1449|   4.5|2009|  Oct|  27|  21|         Tue|
|     1|   1590|   2.5|2009|  Oct|  27|  21|         Tue|
|     1|   1591|   1.5|2009|  Oct|  27|  21|         Tue|
|     1|   2134|   4.5|2009|  Oct|  27|  21|         Tue|
|     1|   2478|   4.0|2009|  Oct|  27|  21|         Tue|
|     1|   2840|   3.0|2009|  Oct|  27|  21|         Tue|
|     1|   2986|   2.5|2009|  Oct|  27|  21|         Tue|
|     1|   3020|   4.0|2009|  Oct|  27|  21|         Tue|
|     1|   3424|   4.5|2009|  Oct|  27|  21|         Tue|
|     1|   3698|   3.5|2009|  Oct|  27|  21|         Tue|
|     1|   382

Convert df_ratings into dataframe df_rating_summary dataset

In [29]:
#Pre-processing
#creating ratings_summary dataframe which contains avg rating and rating count group by movieId

df_ratings_summary=df_ratings.groupby("movieId") \
       .agg(mean("rating").alias("avg_rating"), \
       count("userId").alias("rating_count")) \
       .sort(asc("movieId"))

In [30]:
#displaying ratings_summary
df_ratings_summary.show()


+-------+------------------+------------+
|movieId|        avg_rating|rating_count|
+-------+------------------+------------+
|      1| 3.886452155439497|       22594|
|      2| 3.238923163208076|        8915|
|      3|3.1868289877300615|        5216|
|      4| 2.904433497536946|        1015|
|      5|  3.09004647560031|        5164|
|      6| 3.852959864449857|        9443|
|      7|3.3775669861138273|        5113|
|      8|3.1136363636363638|         506|
|      9| 3.041392904073587|        1522|
|     10|3.4277575205104833|       10970|
|     11| 3.669363538295577|        6489|
|     12|2.6782242384964356|        1543|
|     13| 3.323394495412844|         654|
|     14| 3.460434782608696|        2300|
|     15|2.7338308457711444|        1005|
|     16| 3.802249430523918|        7024|
|     17| 3.956158287226365|        8314|
|     18|3.4132231404958677|        2057|
|     19| 2.642570036540804|        8210|
|     20|2.8929032258064518|        1550|
+-------+------------------+------

Cleaning and pre-processing of df_movies_metadata dataframe - additional metadata

In [31]:
#delete unwanted columns from df_movies_metadata dataframe

df_movies_metadata=df_movies_metadata.drop("genres").drop("id") \
                                    .drop("imdb_id").drop("original_title") \
                                    .drop("overview").drop("belongs_to_collection") \
                                    .drop("poster_path").drop("status") \
                                    .drop("tagline").drop("video").drop("homepage") \
                                    .drop("vote_average").drop("vote_count") \
                                    .drop("revenue").drop("budget").drop("spoken_languages")

Data cleaning is not required for df_movies dataframe
if some data were missing from the dataset we could easily drop them if the percentage were insignificant, Otherwise we could have replaced the values with mean data.

Creating df_movies_ratings_summary_joined dataframe by inner joining df_movies and df_ratings_summary dataframe

In [32]:
#preprocessing
#Join df_movies and df_ratings_summary dataframe

df_movies_ratings_summary_joined = df_movies.join(df_ratings_summary, \
df_movies["movieId"] == df_ratings_summary["movieId"], \
how="inner").drop(df_movies["movieId"])

Pre-processing of df_movies_ratings_summary_joined dataframe

In [33]:
#pre-processing
#remove any leading and lagging spaces using trim function
 
df_movies_ratings_summary_joined = df_movies_ratings_summary_joined \
.withColumn('title', trim(df_movies_ratings_summary_joined.title))

df_movies_ratings_summary_joined.show(truncate =False)

+-------------------------------------+-------------------------------------------+-------+------------------+------------+
|title                                |genres                                     |movieId|avg_rating        |rating_count|
+-------------------------------------+-------------------------------------------+-------+------------------+------------+
|Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1      |3.886452155439497 |22594       |
|Jumanji (1995)                       |Adventure|Children|Fantasy                 |2      |3.238923163208076 |8915        |
|Grumpier Old Men (1995)              |Comedy|Romance                             |3      |3.1868289877300615|5216        |
|Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |4      |2.904433497536946 |1015        |
|Father of the Bride Part II (1995)   |Comedy                                     |5      |3.09004647560031  |5164        |
|Heat (1

In [34]:
#Removing the trailing year from the title column values using substring.

df_movies_ratings_summary_joined = df_movies_ratings_summary_joined \
.withColumn("title",expr("substring(title, 1, length(title)-7)"))


In [35]:
display(df_movies_ratings_summary_joined)


In [36]:
#join df_movies_metadata with df_movies_ratings_summary_joined

df_movies_ratings_summary_movies_metadata_joined = df_movies_ratings_summary_joined.join(df_movies_metadata, \
df_movies_ratings_summary_joined["title"] == df_movies_metadata["title"], \
how="inner").drop(df_movies_ratings_summary_joined["title"])

In [37]:
df_movies_ratings_summary_movies_metadata_joined.count()

28514

In [38]:
display(df_movies_ratings_summary_movies_metadata_joined)

In [39]:
#rearranging column positions of df_all_data_movies dataframe

df_all_data_movies = df_movies_ratings_summary_movies_metadata_joined \
.select("title","movieId","genres","avg_rating","rating_count","adult", \
        "original_language","popularity","production_companies", \
        "production_countries","release_date","runtime")


df_all_data_movies.show()

+--------------------+-------+--------------------+------------------+------------+-----+-----------------+----------+--------------------+--------------------+------------+-------+
|               title|movieId|              genres|        avg_rating|rating_count|adult|original_language|popularity|production_companies|production_countries|release_date|runtime|
+--------------------+-------+--------------------+------------------+------------+-----+-----------------+----------+--------------------+--------------------+------------+-------+
|40 Guns to Apache...|  72163|Adventure|Romance...|3.3333333333333335|           3|False|               en|  1.158998|[{'name': 'Columb...|[{'iso_3166_1': '...|  1967-05-01|   95.0|
|A Brave Heart: Th...| 149534|         Documentary|               3.5|           1|False|               en|  0.161354|                  []|[{'iso_3166_1': '...|  2015-03-14|   90.0|
|   A Cry in the Wild| 121141|Action|Adventure|...|2.6666666666666665|           3|False| 

Using externam dataset for language codes to replace original language codes to actual language by joining df_all_data_movies with df_language_codes.


In [40]:
#replacing language codes with language

df_all_data_movies = df_all_data_movies.join(df_language_codes, \
df_all_data_movies["original_language"] == df_language_codes["code"], \
how="inner").drop(df_all_data_movies["original_language"])

In [41]:
#remove unwanted columns
df_all_data_movies = df_all_data_movies.drop("code")
df_all_data_movies.printSchema()


root
 |-- title: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- rating_count: long (nullable = false)
 |-- adult: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- language: string (nullable = true)



In [42]:
df_all_data_movies=df_all_data_movies.withColumn("runtime",col("runtime").cast("int"))


In [43]:
display(df_all_data_movies)

In [44]:
df_all_data_movies.printSchema()

root
 |-- title: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- rating_count: long (nullable = false)
 |-- adult: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- language: string (nullable = true)



In [45]:

display(df_all_data_movies)

In [46]:

#initialize Counter to 0
i = 0 

#Total records in the df_movies dataframe

total = df_ratings_summary.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(df_ratings_summary.columns)))
print("+++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++")

#while Loop used here to display the count of  each column's missing values 

while i < len(df_ratings_summary.columns):                             
#Loop which checks every column
    print(str(i+1) + "." + str(df_ratings_summary[i]))
    print("  Missing Values = ") 
   
    missingvalue1 = df_ratings_summary \
    .select([count(when(df_ratings_summary[i].isNull(),True))]).show() 
    i = i+1 
    #Counter is incremented by 1

Total Records = 39787
Total columns = 3
+++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++
1.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

2.Column<b'avg_rating'>
  Missing Values = 
+---------------------------------------------------+
|count(CASE WHEN (avg_rating IS NULL) THEN true END)|
+---------------------------------------------------+
|                                                  0|
+---------------------------------------------------+

3.Column<b'rating_count'>
  Missing Values = 
+-----------------------------------------------------+
|count(CASE WHEN (rating_count IS NULL) THEN true END)|
+-----------------------------------------------------+
|                                                 

# Checking data after cleaning and pre-processing

Our final dataframe is df_all_data_movies. This dataframe contains all the required columns data after multiple join operation and data cleaning. This dataframe has only been created for visualization and exploratory data analysis.
Main machine learning model will work on the orimary dataset which is df_ratings dataframe. This dataframe is clean and do not require any pre-processing or cleaning activity. 

In [47]:
#count of records
df_all_data_movies.count()

28389

In [48]:
#print schema details
df_all_data_movies.printSchema()

root
 |-- title: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- rating_count: long (nullable = false)
 |-- adult: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- language: string (nullable = true)



In [49]:

#initialize Counter to 0
i = 0 

#Total records in the df_ratings dataframe

total = df_all_data_movies.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(df_all_data_movies.columns)))
print("+++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++")

#while Loop used here to display the count of  each column's missing values 

while i < len(df_all_data_movies.columns):                             
#Loop which checks every column
    print(str(i+1) + "." + str(df_all_data_movies[i]))
    print("  Missing Values = ") 
   
    missingvalue1 = df_all_data_movies.select([count(when(df_all_data_movies[i].isNull(),True))]).show() 
    i = i+1 
    #Counter is incremented by 1

Total Records = 28389
Total columns = 12
+++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++
1.Column<b'title'>
  Missing Values = 
+----------------------------------------------+
|count(CASE WHEN (title IS NULL) THEN true END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+

2.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

3.Column<b'genres'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (genres IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

4.

# Visualization and EDA

In [50]:
#Rating count by month 
display(df_ratings_final)

In [51]:
#Rating count by days of week 
display(df_ratings_final)

In [52]:
#Rating count by hour of the day
display(df_ratings_final)

In [53]:
#Rating count by month - pie chart
display(df_ratings_final)

In [ ]:
#Average rating vs count of rating
display(df_ratings_summary)

In [55]:
#Movie count by genres

joindf=df_movies.join(df_ratings,['movieId'], how='inner').drop(df_ratings['movieId'])
df_movie_by_genres = df_movies.withColumn("genres", explode(split("genres","[|]")))
df_movie_by_genres.groupBy("genres").count().show()

#movie count where genre is not listed
df_movie_by_genres.filter(df_movies.genres=="(no genres listed)").count()

+------------------+-----+
|            genres|count|
+------------------+-----+
|             Crime| 5105|
|           Romance| 7412|
|          Thriller| 8216|
|         Adventure| 4067|
|             Drama|24144|
|               War| 1820|
|       Documentary| 5118|
|           Fantasy| 2637|
|           Mystery| 2773|
|           Musical| 1113|
|         Animation| 2663|
|         Film-Noir|  364|
|(no genres listed)| 4266|
|              IMAX|  197|
|            Horror| 5555|
|           Western| 1378|
|            Comedy|15956|
|          Children| 2749|
|            Action| 7130|
|            Sci-Fi| 3444|
+------------------+-----+



4266

In [56]:
#movie count by genres

display(df_movie_by_genres)

In [57]:
#released movie count by year - time series  

display(df_all_data_movies)


In [58]:
#histogram of movies by runtime

display(df_all_data_movies)


In [59]:

display(df_all_data_movies)

In [60]:
#describe data statistics of df_all_data_movies dataframe

display(df_all_data_movies.describe().show())

# Machine Learning approach

Spliting df_ratings dataframe into training and test dataset

In [61]:
(training,test)=df_ratings.randomSplit([0.7,0.3])

Alternating Least Squares (ALS) is an approach of matrix factorisation which attempts to estimate the ratings matrix as the product of two lower-rank matrices.It's an iterative process. During each iteration, one of the factor matrices is kept constant, while the other is solved for using least squares. The newly-solved factor matrix is then held constant while solving for the other factor matrix.

ALS can overcome two big challenges-
- Sparsity- Not all items will be rated so there will be lot of blank infrmation in user item matrix.
- Cold start problem- Addition of a new user or a new item where both do not have history in terms of ratings.

Model parameters for ALS are-
- maxIter- number of iterations ALS will do.
- regParam- the regularization parameter in ALS 
- userCol,itemCol and ratingCol are the setting features
- coldStartStrategy - when prediction is run and a model is not trained for a particular user/ ratings or items are not found for a user then it will drop those users if given “drop” as parameter’s value.

In [62]:
als=ALS(maxIter=5,regParam=0.01,userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop")
model=als.fit(training)


In [63]:
predictions=model.transform(test)
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

Py4JJavaError: An error occurred while calling o516.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 287.0 failed 1 times, most recent failure: Lost task 0.0 in stage 287.0 (TID 14149, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:97)
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:120)
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:135)
	at org.apache.spark.io.LZ4CompressionCodec.compressedOutputStream(CompressionCodec.scala:117)
	at org.apache.spark.serializer.SerializerManager.wrapForCompression(SerializerManager.scala:156)
	at org.apache.spark.serializer.SerializerManager.wrapStream(SerializerManager.scala:131)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:120)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237

Let's try hyper parameter tuning-

In [ ]:
# tuning hyper-parameter

als=ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)


# creating a parameter grid for all combination of ranks, max iterations and regularization parameters
param_grid=ParamGridBuilder().addGrid(als.rank,[5,10,15]) \
.addGrid(als.maxIter,[5,10,15]).addGrid(als.regParam,[0.01,0.05,0.1]).build()






In [ ]:
#calculating rmse 
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
tvs=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)

#trainig the model with training data
model=tvs.fit(training)

In [ ]:
#calculating RMSE using best model

best_model=model.bestModel
predictions=best_model.transform(test)
rmse=evaluator.evaluate(predictions)

In [ ]:
#printing RMSE, Rank, Max interation and regularization parameter using best model

print("RMSE="+str(rmse))
print("Rank="+str(best_model.rank))
print("MaxIter="+str(best_model._java_obj.parent().getMaxIter()))
print("RegParam="+str(best_model._java_obj.parent().getRegParam()))

Clearly best model out performed our previous model as RMSE dropped.

Recommendation for all userId

In [ ]:

a=best_model.recommendForAllUsers(10).sort("userId")


Comparison of prediction vs actual.

In [ ]:
predictions.sort("userId","rating").show()

Well all user recomendation output is difficult to expain because of its format, Let's create a function to make it more explainable

In [ ]:

def get_rec(rec):
  rec=rec.select("userId",explode(rec.recommendations))
  rec=rec.select("col.*","*")
  rating_mat=rec.select(["userId","movieId","rating"])
  return rating_mat

In [ ]:
rating_mat=get_rec(best_model.recommendForAllUsers(8))

In [ ]:
rating_mat.show()

Lets merge this result with the movie data to get a clear picture

In [ ]:
final_df=rating_mat.join(df_movies,['movieId'],how='left')

In [ ]:
final_df.filter("userId=10").orderBy(final_df["rating"].desc()).show(10)

Now lets merge the rating data with the movie data and compare the result outcome with the actuals for a random user

In [ ]:
df_test=df_movies.join(df_ratings,['movieId'],how='left')
df_test.filter("userId=10").orderBy(df_test["rating"].desc()).show(10)

As we can see user 5 mostly liked comedy, romance and drama movies in actual and in our recomendation also he is mostly getting comedy movie suggestions.

In [ ]:
#



predictions.join(df_movies,"movieId").select("userId","title","genres","prediction").show(5)

In [ ]:
#showing movie recommendation for perticular user and links by joining the df_linkd dataframe with predictions 

for_one_user=predictions.filter(col("userId")==10).join(df_movies, "movieId") \
.join(df_links, "movieId").select("userId","title","genres","prediction","tmdbId",)
display(for_one_user)

In [ ]:
#generating top three movie recommendations and the links for the movie homepage in TMDB
url="https://www.themoviedb.org/movie/"
for movie in for_one_user.take(3):
    movieURL=url+str(movie.tmdbId)
    print(movieURL)
    print(movie.title)
    

In [ ]:
#top 5 movie recommendation for each user

userRecommends = best_model.recommendForAllUsers(5)

userRecommends.select("userId","recommendations.movieId").show(10,False)


In [ ]:
#top 5 user recommendation for each movie

itemRecommends = best_model.recommendForAllItems(5)

itemRecommends.select("movieId","recommendations.userId").show(10,False)

# Popularity based recommendation when there are no prior existance of the user or item(movie)

In [ ]:
#General popular movies

popular_movies=df_all_data_movies.sort("popularity")
popular_movies.show()